# Расчет беты

In [ ]:
#Импорт библиотек
import pandas as pd
import numpy as np
from scipy.stats import mstats
import os
import gc
import matplotlib
import matplotlib.pyplot as plt
import statsmodels as sm
import statsmodels.api as smp
import statsmodels.formula.api as smf
%matplotlib inline  

Загрузка и обработка эксель-файла

In [ ]:
#Сырые данные
close = pd.read_csv('C:/Py/final_data/close.csv', sep=';') # Цена закрытия
vtb = pd.read_csv('C:/Py/final_data/vtb.csv', sep=';') # Отдельно для ВТБ, так как у блумберга недостаточно точные значения
micex = pd.read_csv('C:/Py/Disser_data/micex.csv', sep=';') # Индекс
date_control = pd.read_csv('C:/Py/final_data/control/date_control.csv', sep=';')
months_control = pd.read_csv('C:/Py/final_data/control/months_control.csv', sep=';') # понадобится для создания портфелей по месяцам
#Необходимо выбрать значения переменных в торговые дни. Для этого используется отдельная таблица Data_control 
date_control = date_control[['Data']].dropna() 
#Соединяем таблицы по общим значениям (inner join):
close_values = pd.merge(close, date_control, how='inner', on=['Data']).drop('Data', 1)
micex = pd.merge(micex, date_control, how='inner', on=['Data']).drop('Data', 1)
vtb = pd.merge(vtb, date_control, how='inner', on=['Data']).drop('Data', 1)
# Добавляем доходность индекса в цены закрытия:
close_values['MICEX'] = micex['MICEX']
close_values['VTBR RM Equity'] = vtb
#Окно для расчета беты:
window = 247

Преобразование переменных

In [ ]:
#Записываем функцию для нахождения доходности. Представим доходности как отношения двух таблиц, отличающихся датой
# Для дневных - дата смещается на один день. Для годовых - на год
def day_returns(df):
    df_1 = df.drop(0,0).reset_index(drop=True)
    df_2 = df.drop(len(df)-1,0).reset_index(drop=True)
    df_3 = pd.DataFrame(np.log(df_1.values) - np.log(df_2.values), columns=df_1.columns, index=df_1.index)
    return df_3

def year_returns(df):   
    df_1 = df.ix[window:,:].reset_index(drop=True)
    df_2 = df.ix[0:len(df)-window-1,:].reset_index(drop=True)
    df_3 = pd.DataFrame(np.log(df_1.values) - np.log(df_2.values), columns=df_1.columns, index=df_1.index)
    return df_3

def month_returns(df, months_control):
    d = pd.merge(df, months_control, how='inner', on=['Data']) # оставляем только месячные наблюдения
    d = d.drop('Data', 1)
    df_1 = d.drop(0,0).reset_index(drop=True)
    df_2 = d.drop(len(d)-1,0).reset_index(drop=True)
    df_3 = pd.DataFrame(np.log(df_1.values) - np.log(df_2.values), columns=df_1.columns, index=df_1.index)
    df_3 = df_3.ix[11:len(df_3), :].reset_index(drop=True)
    return df_3
# Получаем доходности 
close_returns = day_returns(close_values)
close_returns_y=year_returns(close_values)
close_returns_m = month_returns(close, months_control)
# Добавляем дату, чтобы выбрать отсюда месячные наблюдения
close_returns_y['Data']=date_control['Data'][window:len(date_control)].reset_index(drop=True)
close_returns_y_m = pd.merge(close_returns_y, months_control, how = 'inner', on = 'Data')


Расчет беты

In [ ]:
def d_betas_func(df,window):
    d_betas = pd.DataFrame()
    for i in range(0, (len(df)-window)): # петля по окну
        d = df.ix[i:(window+i),:]
        micex_mean = d['MICEX'].mean()
        d = d.ix[(d['MICEX']<micex_mean)] # выбираем наблюдения, где рыночная доходность меньше нуля
        market_variance = np.var(d['MICEX']) # считаем полудисперсию рынка
        betas = pd.DataFrame()
        for column in d:
            beta = np.cov(d[column], d['MICEX'])[0,1]/market_variance
            betas[column]=pd.Series(beta)
        d_betas = d_betas.append(betas, ignore_index=True)
    gc.collect()
    return d_betas

def d_est_betas_func(df,window):
    d_betas = pd.DataFrame()
    for i in range(0, (len(df)-window)): # петля по окну
        d = df.ix[i:(window+i),:]
        d = d.ix[(d['MICEX']<d['MICEX'].mean())] # выбираем наблюдения, где рыночная доходность меньше среднего
        market_variance = np.var(d['MICEX']) # считаем полудисперсию рынка
        betas = pd.DataFrame()
        for column in d:
            d_s = d.ix[(d[column]<d['MICEX'].mean())] # выбираем наблюдения, где доходность компании меньше среднего
            beta = np.cov(d_s[column], d_s['MICEX'])[0,1]/market_variance
            betas[column]=pd.Series(beta)
        d_betas = d_betas.append(betas, ignore_index=True)
    gc.collect()
    return d_betas

def up_betas_func(df,window):
    up_betas = pd.DataFrame()
    for i in range(0, (len(df)-window)): # петля по окну
        d=df.ix[i:window+i,:]
        micex_mean = d['MICEX'].mean()
        d = d.ix[(df['MICEX']>micex_mean)] # выбираем наблюдения, где рыночная доходность меньше нуля
        market_variance = np.var(d['MICEX']) # считаем полудисперсию рынка
        betas = pd.DataFrame()
        for column in d:
            beta = np.cov(d[column], d['MICEX'])[0,1]/market_variance
            betas[column]=pd.Series(beta)
        up_betas = up_betas.append(betas, ignore_index=True)
    gc.collect()
    return up_betas

def capm_betas_func(df,window):
    capm_betas = pd.DataFrame()
    for i in range(0, (len(df)-window)): # петля по окну
        d=df.ix[i:window+i,:]
        market_variance = np.var(d['MICEX']) # считаем дисперсию рынка
        betas = pd.DataFrame()
        for column in d:
            beta = np.cov(d[column], d['MICEX'])[0,1]/market_variance
            betas[column]=pd.Series(beta)
        capm_betas = capm_betas.append(betas, ignore_index=True)
    gc.collect()
    return capm_betas

In [ ]:
# Записываем беты в один словарь
beta = ['d_est_betas', 'd_betas', 'up_betas', 'capm_betas']
frames={}
for b in beta:
    frames[b] = globals()[b +'_func'](close_returns, window)

Расчет спреда между бетами

In [ ]:
frames['beta_spread'] = pd.DataFrame(frames[beta[1]]-frames[beta[2]], columns=frames[beta[1]].columns, index=frames[beta[1]].index)
frames['beta_spread_d'] = pd.DataFrame(frames[beta[1]]-frames[beta[3]], columns=frames[beta[1]].columns, index=frames[beta[1]].index)
frames['beta_spread_up'] = pd.DataFrame(frames[beta[2]]-frames[beta[3]], columns=frames[beta[1]].columns, index=frames[beta[1]].index)
def handeling(df):
    d = df
    d['Data'] = date_control['Data'][window:len(date_control)].reset_index(drop=True)
    d = pd.merge(d, months_control, how = 'inner', on = 'Data')
    d = d.drop(['MICEX','Data'], 1)
    return d
frames_m = {}
for k, df in frames.iteritems():
    frames_m[k] = handeling(df)
# Проводим Windworizing выборки на уровне беты и доходности (для всех периодов). Заменяем значения выше(ниже) 95(05) квантиля
def winsor(df):
    d = pd.DataFrame(columns=df.columns)
    for t in range(0, len(df)):
        win = mstats.winsorize(df.loc[t], limits=[0.05, 0.05])
        win_d = pd.DataFrame(np.asarray(win).reshape(1,-1), columns= df.columns)
        d = d.append(win_d, ignore_index = True)
    return d

In [ ]:
# без винзоризации
close_returns_y_m= close_returns_y_m[frames_m['capm_betas'].columns]
close_returns_m= close_returns_m[frames_m['capm_betas'].columns]
frames_return = {}
frames_return['close_returns'] = (close_returns)
frames_return['close_returns_m'] = (close_returns_m)
frames_return['close_returns_y_m'] = (close_returns_y_m)
frames_beta = {}
for k, df in frames_m.iteritems():
    frames_beta[k] = (df)
# Записываем данные
interim_path = 'C:/Py/disser_output/interim_2_without_winsor/betas/'
for k, df in frames_beta.iteritems():
     df.to_csv(interim_path+k+'.csv', index = False, sep = ',' )    
interim_path = 'C:/Py/disser_output/interim_2_without_winsor/returns/'
for k, df in frames_return.iteritems():
     df.to_csv(interim_path+k+'.csv', index = False, sep = ',' )

In [ ]:
beta_mean = []
for k, df in frames_beta.iteritems():
    beta_mean.append(df.mean().mean())

In [ ]:
# с винзоризацией
close_returns_y_m= close_returns_y_m[frames_m['capm_betas'].columns]
close_returns_m= close_returns_m[frames_m['capm_betas'].columns]
frames_return = {}
frames_return['close_returns'] = winsor(close_returns)
frames_return['close_returns_m'] = winsor(close_returns_m)
frames_return['close_returns_y_m'] = winsor(close_returns_y_m)
frames_beta = {}
for k, df in frames_m.iteritems():
    frames_beta[k] = winsor(df)
# Записываем данные
interim_path = 'C:/Py/disser_output/interim_2/betas/'
for k, df in frames_beta.iteritems():
     df.to_csv(interim_path+k+'.csv', index = False, sep = ',' )    
interim_path = 'C:/Py/disser_output/interim_2/returns/'
for k, df in frames_return.iteritems():
     df.to_csv(interim_path+k+'.csv', index = False, sep = ',' )